# Random Sampling in a Hyperplane Arrangement Complement

### Import Packages

In [1]:
using LinearAlgebra
include("lambda_cyclic.jl")

getRank (generic function with 1 method)

### Generate a random point

In [2]:
p(n) = [rand(Float64)-.5 for j=1:n]

p (generic function with 1 method)

### Generate the 3-Sunlet hyperplanes

In [3]:
function getH(k)
    H = []
    v = zero(rand(2*k))
    v[1] = 1
    push!(H, v)
    
    for i=2:k
        for j=(k+1):(2*k)
            v = zero(rand(2*k))
            v[i] = v[j] = 1
            push!(H, v)
            
            w = zero(rand(2*k))
            w[i] = 1
            w[j] = -1
            push!(H, w)
        end
    end
    return H
end

getH (generic function with 1 method)

In [4]:
H3 = [[1 0 0 0 0 0], [0 1 0 -1 0 0], [0 1 0 0 -1 0], [0 1 0 0 0 -1], [0 0 1 1 0 0], [0 0 1 0 1 0], [0 0 1 0 0 1]];
H3

7-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 1 … 0 0]
 [0 1 … -1 0]
 [0 1 … 0 -1]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 1]

In [5]:
coords_3 = ["mu_0", "mu_1", "mu_2", "eta_{10}", "eta_{21}", "eta_{02}"]
coords_3[[i for i in eachindex(H3[1]) if H3[1][i] != 0]]

1-element Vector{String}:
 "mu_0"

In [6]:
for h in eachindex(H3)
    H = H3[h]
    print("\$H_", h, "\$ & ")
    for j in eachindex(H)
        print(H[j], " & ")
    end
    vars = [i for i in eachindex(H) if H[i] != 0]
    coord = coords[vars]
    coeff = H[vars]
    print("\$")
    for v in eachindex(vars)
        if v != length(vars)
            if coeff[v+1] > 0
                print("\\", coord[v], " + ")
            else
                print("\\", coord[v], " - ")
            end
        else
            print("\\", coord[v], " = 0\$ \\")
        end
    end
    println()
end

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 

LoadError: UndefVarError: coords not defined

In [7]:
coords_4 = ["mu_0", "mu_1", "mu_2", "mu_3", "eta_{10}", "eta_{21}", "eta_{32}", "eta_{03}"]

8-element Vector{String}:
 "mu_0"
 "mu_1"
 "mu_2"
 "mu_3"
 "eta_{10}"
 "eta_{21}"
 "eta_{32}"
 "eta_{03}"

In [8]:
for h in eachindex(H3)
    H = H3[h]
    print("\$H_", h, "\$ & ")
    for j in eachindex(H)
        print(H[j], " & ")
    end
    vars = [i for i in eachindex(H) if H[i] != 0]
    coord = coords[vars]
    coeff = H[vars]
    print("\$")
    for v in eachindex(vars)
        if v != length(vars)
            if coeff[v+1] > 0
                print("\\", coord[v], " + ")
            else
                print("\\", coord[v], " - ")
            end
        else
            print("\\", coord[v], " = 0\$ \\")
        end
    end
    println()
end

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 

LoadError: UndefVarError: coords not defined

In [28]:
function HToLatex(Hs, coords)
    for h in eachindex(Hs)
        H = Hs[h]
        print("\$H_", h, "\$ & ")
        for j in eachindex(H)
            print(H[j], " & ")
        end
        
        vars = [i for i in eachindex(H) if H[i] != 0]
        coord = coords[vars]
        coeff = H[vars]
        print("\$")
        for v in eachindex(vars)
            if v != length(vars)
                if coeff[v+1] > 0
                     print("\\", coord[v], " + ")
                else
                    print("\\", coord[v], " - ")
                end
            else
                print("\\", coord[v], " = 0\$ \\")
            end
        end
        println()
    end
end

HToLatex (generic function with 1 method)

In [10]:
HToLatex(H4, coords_4)

LoadError: UndefVarError: H4 not defined

In [11]:
function generateSunletArr(groupStructure)
    groupSize = prod(groupStructure)
    validTriples = getValidGroupTriples(groupStructure)
    groupElements = vec(getGroupElements(groupStructure))
    numFactors = length(groupStructure)
    
    H = []
    
    mu0 = zeros(Int, 2*groupSize)
    mu0[1] = Int(1)
    push!(H, mu0)
    
    for G in validTriples
        if G[1] != zeros(Int, numFactors)
            ## get the mu index
            muIndex = findfirst(x -> x == G[1], groupElements)
            etaIndex = [findfirst(x -> x == ((G[1] .+ G[2]) .% groupStructure), groupElements), findfirst(x -> x == G[2], groupElements)]
            
            ## also need to determine the sign of eta
            ## +1 if g_1 + g_2 index is higher than g_2 index, -1 otherwise
            etaSign = Int((Int(etaIndex[1] > etaIndex[2])-.5)*2)
            
            ## convert eta index if necessary
            if Set(etaIndex) == Set([1, groupSize])
                etaIndex = groupSize
                etaSign = -etaSign
            elseif abs(etaIndex[1] - etaIndex[2]) > 1
                newEtaIndex = [i for i=(minimum(etaIndex)+1):maximum(etaIndex)]
                etaIndex = newEtaIndex
            else
                etaIndex = maximum(etaIndex)-1
            end
            
            ##                        1  2  3  4  5       n
            ## eta coordinate order: 10 21 32 43 54 ... 0(n-1)
            newHyperplane = [Int(i-groupSize in etaIndex)*etaSign for i=1:2*groupSize]
            newHyperplane[muIndex] = 1
            push!(H, newHyperplane)
        end
    end
    return H
end

generateSunletArr (generic function with 1 method)

In [12]:
H5 = generateSunletArr([5])
sort(H5)

21-element Vector{Any}:
 [0, 0, 0, 0, 1, -1, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, -1, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, -1, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, -1, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, -1]
 [0, 0, 0, 1, 0, 0, -1, -1, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, -1, -1, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, -1, -1]
 [0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
 [0, 0, 0, 1, 0, 0, 1, 1, 1, 0]
 [0, 0, 1, 0, 0, 0, -1, -1, -1, 0]
 [0, 0, 1, 0, 0, 0, 0, -1, -1, -1]
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 1]
 [0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Inequality vector

In [13]:
ineq(v, H) = [LinearAlgebra.dot(v, h) > 0 for h in H]

ineq (generic function with 1 method)

In [14]:
neighbors(I, J) = (sum(I .⊻ J) == 1)

neighbors (generic function with 1 method)

### Sampling Algorithm

In [15]:
"""
H is the hyperplane arrangement
N is the number of samples to take
groupStructure is a list of factors for the finite abelian group
"""
function sample(groupSize::Int, H, N::Int)
    sample = Dict()
    for i=1:N
        v = p(2*groupSize)
        v[end] = -sum(v[(groupSize+1):(end-1)]) ## making sure the condition on the sum of the lambda holds
        sample[ineq(v, H)] = v
    end
    return sample
end

sample

## Example $n = 3$

In [17]:
n = 3
@time s3 = sample(3, H3, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s3))
ranks = [getRank(s, n) for s in collect(values(s3))]; ## get the rank for each sample point
R = Set(ranks)
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

  0.479135 seconds (3.15 M allocations: 256.006 MiB, 9.36% gc time)
92
R=Set([7, 9, 8])


3-element Vector{Int64}:
  4
 64
 24

## checking if $\mu_0$ does anything...

In [46]:
inequalityKeys = collect(keys(s3))
for I in inequalityKeys
    for J in inequalityKeys
#         println(I .⊻ J)
        if I .⊻ J == [Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)]
            print(getRank(s3[I], [3]) - getRank(s3[J], [3]))
        end
    end
end

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [44]:
print([Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)])

Bool[1, 0, 0, 0, 0, 0]

### Test Neighbors

In [15]:
for I in collect(keys(s3))
    s = s3[I]
    curr_rank = getRank(s, n)
    for J in collect(keys(s3))
        if neighbors(I, J)
            neighbor = s3[J]
            n_rank = getRank(neighbor, n)
            print(abs(curr_rank - n_rank))
        end
    end
end

10010001011111011110111100000101101111000000000000001101111110110000000000000011110110100000111101111101111010010100110111101001000100011001101001011000100010000000110001111011111100000000011001111101011000110011000011001111101000000000000000010101111110010100101001010111100000000111110101000000001111100010110100011111101111010010010111011100000001001001111100000000111110010110100101000111110010100000000000000110111100011110000001100011111101100000000001110111111101111000000000011000110001010000011001101100010010111011

### Parochial Chambers

In [16]:
p1 = s3[[1, 1, 1, 1, 1, 1, 1]];
p2 = s3[[0, 0, 0, 0, 0, 0, 0]];
r1 = getRank(p1, n);
r2 = getRank(p2, n);
println(r1, r2)
parochial(s3, 7)
# keys(s3)

77


LoadError: UndefVarError: parochial not defined

In [17]:
function parochial(S, k)
    p0 = S[zeros(Bool, k)]
    p1 = S[ones(Bool, k)]
    r0 = getRank(p0, n)
    r1 = getRank(p1, n)
    print(r0, ' ', r1)
end

parochial (generic function with 1 method)

In [18]:
p_avg = (p1 .+ p2)./2; ## average the sample representing parochial chambers
r_avg = getRank(p_avg, n) ## the average happens to have full rank

8

### Low Rank Chambers

In [19]:
low_rank = []
s3_keys = collect(keys(s3))
for s in eachindex(s3_keys)
    if ranks[s] == 7
        push!(low_rank, s)
    end
end
length(low_rank)

4

In [20]:
## two of the chambers are parochial
s3_keys[low_rank]

4-element Vector{Any}:
 Bool[1, 1, 1, 1, 1, 1, 1]
 Bool[1, 0, 0, 0, 0, 0, 0]
 Bool[0, 1, 1, 1, 1, 1, 1]
 Bool[0, 0, 0, 0, 0, 0, 0]

### Testing to_lambda function

In [21]:
s0 = collect(values(s3))[1];
ineq(s0, H3)

7-element Vector{Bool}:
 0
 0
 0
 1
 1
 1
 1

In [22]:
G = [[0, 2, 1], [1, 1, 1], [1, 0, 2], [1, 2, 0], [2, 2, 2], [2, 1, 0], [2, 0, 1]]
for g in G
    print(computeLambda(g, m1, to_lambda(s0[1:3], s0[4:6], 3), 3), ' ')
    println(computeLambda(g, m2, to_lambda(s0[1:3], s0[4:6], 3), 3))
end

LoadError: UndefVarError: m1 not defined

In [23]:
s0

6-element Vector{Float64}:
 -0.37278034640847113
 -0.193400337932362
  0.38174121635480174
 -0.13205504763953946
  0.42126038943258004
 -0.2892053417930406

## Example $n = 4$

### Generate the Hyperplanes

In [49]:
## mu 0 1 2 3 eta 10 21 32 03
H4 = [[1 0 0 0 0 0 0 0], [0 0 1 0 0 -1 -1 0], [0 0 1 0 -1 -1 0 0], [0 0 1 0 0 1 1 0], [0 0 1 0 1 1 0 0], [0 1 0 0 0 0 -1 0], [0 1 0 0 0 -1 0 0], [0 1 0 0 -1 0 0 0], [0 1 0 0 0 0 0 -1], [0 0 0 1 0 0 0 1], [0 0 0 1 0 0 1 0], [0 0 0 1 0 1 0 0], [0 0 0 1 1 0 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 1 … -1 0]
 [0 1 … 0 0]
 [0 1 … 0 0]
 [0 1 … 0 -1]
 [0 0 … 0 1]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … 0 0]

### Generate Sample Points

In [25]:
## 2328

In [47]:
function sample_info(n, H, N)
    @time S = sample(n, H, N)
    println(length(S))
    ranks = [getRank(s, n) for s in collect(values(S))];
    R = sort(collect(Set(ranks)))
    println("R=", R)
    counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
    println(counts)
    priors = [100*c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
    println(priors)
    return S
end

sample_info (generic function with 1 method)

In [50]:
s4 = sample_info(4, H4, 2^20);

  1.113067 seconds (3.15 M allocations: 272.111 MiB, 8.59% gc time)
2328
R=[10, 11, 12, 13, 14, 15, 16]
[4, 48, 180, 496, 864, 624, 112]
[0.1718213058419244, 2.0618556701030926, 7.731958762886598, 21.305841924398624, 37.11340206185567, 26.804123711340207, 4.810996563573883]


In [52]:
inequalityKeys = collect(keys(s4))
for I in inequalityKeys
    for J in inequalityKeys
#         println(I .⊻ J)
        if I .⊻ J == [Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)]
            print(getRank(s4[I], [3]) - getRank(s4[J], [3]))
        end
    end
end

00-10000-100000000001010-10001010-10000000000-2100-10000000201-1000-100-1-111000000010000-110000-100000000010000010-1-1100100-2-2000000000100000-100-100100-10020000-1200-110000000000-10-1000-10010000000011-20000-100010-1-10-10010001100-1000000101000000000100-200000-1001-10-10000000010100001000-10-100010000000-11011010000-100010000020010000010-200010000000-1000-1000-1000000002-1200000000000-10000000000000000-10000000-1000-1100000000000000-101000-10-1000-111000000000100-10000000100001000-1000000100-101-102000-102000-10000-1-100010-10101-1000000000001000000000000010000100001-10000-102000-1110100-1000111-1-1-100-1000010000-100000-100100-1-10200010000-10000000-121000-10001000010-1000100100-101-10-10-1000-11100-100000-10100000000-2-100-10-1-1000000000000001000000-1000-20-200000000000110000-1-1000000010-10000-100010-12000000-11000001-10-1100000010000100-1100000000000-1000-10100000010-10-1000001000001-100000-2000010-1010200-10101000-1-10000000-10000000000000000000100-1-1000000201000101-10-1010

### Check Neighbors

In [24]:
function check_neighbors(S, groupStructure)
    for I in collect(keys(S))
        s = S[I]
        curr_rank = getRank(s, groupStructure)
        for J in collect(keys(S))
            if neighbors(I, J)
                neighbor = S[J]
                n_rank = getRank(neighbor, groupStructure)
                diff = abs(curr_rank - n_rank)
                if diff > 1
                    print(diff)
                end
            end
        end
    end
end

check_neighbors (generic function with 1 method)

In [29]:
check_neighbors(s4, 4)

### Check Parochial Chambers

In [30]:
p0 = s4[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]];
p1 = s4[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]];
r0 = getRank(p0, 4)
r1 = getRank(p1, 4)
print(r0, ' ', r1)

10 10

### Low Rank Chambers

In [31]:
low_rank_4 = []
for s in collect(keys(s4))
    if getRank(s4[s], 4) == 10
        push!(low_rank_4, s)
    end
end
length(low_rank)

4

In [32]:
low_rank_4

4-element Vector{Any}:
 Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Bool[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Bool[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### Dictionary of ranks => inequalities

In [33]:
rdict_3 = Dict([r => [s3_keys[s] for s in eachindex(collect(keys(s3))) if ranks[s] == r] for r in R])

Dict{Int64, Vector{Vector{Bool}}} with 3 entries:
  7 => [[1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1], [0…
  9 => [[0, 0, 0, 1, 1, 1, 1], [1, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1], [1…
  8 => [[1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1], [1…

In [34]:
length(rdict_3[9])

64

In [35]:
function rankToIDict(S, R)
    rToI = Dict()
    K = collect(keys(S))
    for r in Set(R)
        rToI[r] = [K[s] for s in eachindex(K) if R[s] == r]
    end
    return rToI
end

rankToIDict (generic function with 1 method)

In [36]:
rdict_4 = rankToIDict(s4, [getRank(s, 4) for s in collect(values(s4))])

Dict{Any, Any} with 7 entries:
  13 => Vector{Bool}[[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1…
  16 => Vector{Bool}[[1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0…
  15 => Vector{Bool}[[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0…
  11 => Vector{Bool}[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0…
  10 => Vector{Bool}[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0…
  12 => Vector{Bool}[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1…
  14 => Vector{Bool}[[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0…

In [80]:
for r in collect(keys(rdict_4))
    println(length(rdict_4[r]))
end

496
112
624
48
4
180
864


In [81]:
function printRanks(RI)
    for r in sort(collect(keys(RI)))
        println("## rank = ", r)

        count = 0
        for I in RI[r]
            if count % 5 == 0
                println()
            end
            for b in I
                print(Int(b), ' ')
            end
            println()
            count += 1
        end

        println()
    end
end

printRanks (generic function with 1 method)

In [120]:
function latexRanks(RI)
    count = 1
    for r in sort(collect(keys(RI)))
        for I in RI[r]
            print(count, " & ")
            for i in eachindex(I)
                if i < length(I)
                    print(Int(I[i]), " & ")
                else
                    print(Int(I[i]), " & ", r, " & \\\\")
                end
            end
            println()
            count += 1
        end
    end
end

latexRanks (generic function with 1 method)

In [121]:
latexRanks(rdict_3);

1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 7 & \\
2 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & \\
3 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 7 & \\
4 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & \\
5 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 8 & \\
6 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 8 & \\
7 & 1 & 1 & 1 & 1 & 1 & 0 & 1 & 8 & \\
8 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 8 & \\
9 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 8 & \\
10 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 8 & \\
11 & 1 & 1 & 1 & 1 & 0 & 1 & 1 & 8 & \\
12 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 8 & \\
13 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 8 & \\
14 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 8 & \\
15 & 0 & 1 & 1 & 0 & 1 & 1 & 1 & 8 & \\
16 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 8 & \\
17 & 0 & 1 & 1 & 1 & 1 & 0 & 1 & 8 & \\
18 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 8 & \\
19 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 8 & \\
20 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 8 & \\
21 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 8 & \\
22 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 8 & \\
23 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 8 & \\
24 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 8 & \\
25 & 1 & 1 & 0 & 1 & 1 & 1 & 1 & 8 & \\
26 & 1 & 

## Z2 x Z2

In [32]:
## mu 0 1 2 3, eta 10 21 32 03
H22 = [[1 0 0 0 0 0 0 0], [0 1 0 0 1 0 0 0], [0 1 0 0 0 0 1 0], [0 1 0 0 -1 0 0 0], [0 1 0 0 0 0 -1 0], [0 0 1 0 1 1 0 0], [0 0 1 0 0 1 1 0], [0 0 1 0 -1 -1 0 0], [0 0 1 0 0 -1 -1 0], [0 0 0 1 0 0 0 1], [0 0 0 1 0 1 0 0], [0 0 0 1 0 0 0 -1], [0 0 0 1 0 -1 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 1 … 0 0]
 [0 1 … 1 0]
 [0 1 … 0 0]
 [0 1 … -1 0]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 1]
 [0 0 … 0 0]
 [0 0 … 0 -1]
 [0 0 … 0 0]

In [33]:
groupStructure = [2,2]
@time s22 = sample(4, H22, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s22))

  0.744872 seconds (3.15 M allocations: 272.113 MiB, 9.39% gc time)
2328


In [35]:
ranks = [getRank(s, [2,2]) for s in collect(values(s22))]; ## get the rank for each sample point
R = sort(collect(Set(ranks)))
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

R=[10, 11, 12, 13, 14, 15]


6-element Vector{Int64}:
    4
   48
  156
  584
 1056
  480

In [36]:
priors

6-element Vector{Float64}:
 0.001718213058419244
 0.020618556701030927
 0.06701030927835051
 0.2508591065292096
 0.4536082474226804
 0.20618556701030927

In [37]:
check_neighbors(s22, [2,2])

In [34]:
HToLatex(H22, coords_4)

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & $\mu_0 = 0$ \
$H_2$ & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & $\mu_1 + \eta_{10} = 0$ \
$H_3$ & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & $\mu_1 + \eta_{32} = 0$ \
$H_4$ & 0 & 1 & 0 & 0 & -1 & 0 & 0 & 0 & $\mu_1 - \eta_{10} = 0$ \
$H_5$ & 0 & 1 & 0 & 0 & 0 & 0 & -1 & 0 & $\mu_1 - \eta_{32} = 0$ \
$H_6$ & 0 & 0 & 1 & 0 & 1 & 1 & 0 & 0 & $\mu_2 + \eta_{10} + \eta_{21} = 0$ \
$H_7$ & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 0 & $\mu_2 + \eta_{21} + \eta_{32} = 0$ \
$H_8$ & 0 & 0 & 1 & 0 & -1 & -1 & 0 & 0 & $\mu_2 - \eta_{10} - \eta_{21} = 0$ \
$H_9$ & 0 & 0 & 1 & 0 & 0 & -1 & -1 & 0 & $\mu_2 - \eta_{21} - \eta_{32} = 0$ \
$H_10$ & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & $\mu_3 + \eta_{03} = 0$ \
$H_11$ & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & $\mu_3 + \eta_{21} = 0$ \
$H_12$ & 0 & 0 & 0 & 1 & 0 & 0 & 0 & -1 & $\mu_3 - \eta_{03} = 0$ \
$H_13$ & 0 & 0 & 0 & 1 & 0 & -1 & 0 & 0 & $\mu_3 - \eta_{21} = 0$ \


## Z/5Z

In [20]:
@time s5 = sample(5, H5, 2^30); ## get the sample points (< 2 seconds usually)
println(length(s5))

1889.224344 seconds (27.92 G allocations: 704.007 GiB, 14.22% gc time)
113349


In [210]:
# 113350

In [57]:
ranks = [getRank(s, [5]) for s in collect(values(s5))]; ## get the rank for each sample point
R = sort(collect(Set(ranks)))
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

R=[13, 14, 15, 16, 17, 18, 19, 20, 21]


9-element Vector{Int64}:
    10
   268
   956
  2598
  4922
 11094
 33520
 40308
 19673

In [23]:
priors

9-element Vector{Float64}:
 8.822309857166804e-5
 0.0023643790417207035
 0.008434128223451465
 0.022920361008919355
 0.04342340911697501
 0.09787470555540852
 0.29572382641223127
 0.3556096657226795
 0.17356130182004254

In [26]:
check_neighbors(s5, [5])

2322423232422223222222442232322222232344222224222222222222222222322244435432332332222222222222222222322222323222222222222232223223222222224424322322223222222222243334344322222332222222222322223232222222222222322233222222322222222222224223322222222322222222232222222222222222222222222222232242222222222222222223222222453243522322232422222322232222222423222222222222422333333222343333222222223322232222322222422222423222222242232224223322222222222322232232222232222222222222223233222223222222222232222344234333222222343222232422232222222222222463646632323322232222223232322322223222222222232222223222222222232222222222242322223342323323222422222222222222223232332222222222222222222422222222222222222223222222222232432322222222222222322222322223222222323222222323323222322222222223322222232322222222322334222223222323232223223222222222222432243242222223222232222222222222222222222222222222342344232224222222222222222222422223223222222223232222232223222222223222222222222232242222322223223222222222223233

3222222232222222232222232322222444222233232322423422233432222232352223222222223224322222222222224224222333433232222222222222233342333223222332222222223232322423323322232223332322242223222222232223222222222222224322222222322222232222222222224222322222243522223222222222223222222222222222222224222223242222222234233224223222222232222222222323222222322222223223444245332332222222222223222222222222234253222222222222222264554442423222222222222222222222223222222222222222222223222222222322333322222223332222222222222222322222222232223332322222243233222232242232222232224222233223232222222222422222222222222222222232222222222332322334242222223233232222222222333423222322223222222232222223442224223222232232234332422222322222323222222222222222232222222222222252322332222243323242322222232222224332222223323222222222222422322222222222222222222242222222222222422222222243222222322223222222222343222233224222222222222233233222222343222233332222222222222222322232232222223223222222222322223222222223222222222222

LoadError: InterruptException:

In [30]:
coords_5 = ["mu_0", "mu_1", "mu_2", "mu_3", "mu_4", "eta_{10}", "eta_{21}", "eta_{32}", "eta_{43}", "eta_{04}"]
HToLatex(H5, coords_5)

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & $\mu_0 = 0$ \
$H_2$ & 0 & 0 & 0 & 0 & 1 & -1 & 0 & 0 & 0 & 0 & $\mu_4 - \eta_{10} = 0$ \
$H_3$ & 0 & 0 & 0 & 1 & 0 & 0 & -1 & -1 & 0 & 0 & $\mu_3 - \eta_{21} - \eta_{32} = 0$ \
$H_4$ & 0 & 0 & 1 & 0 & 0 & 0 & -1 & -1 & -1 & 0 & $\mu_2 - \eta_{21} - \eta_{32} - \eta_{43} = 0$ \
$H_5$ & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & $\mu_1 + \eta_{04} = 0$ \
$H_6$ & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -1 & $\mu_4 - \eta_{04} = 0$ \
$H_7$ & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 1 & 1 & $\mu_3 + \eta_{32} + \eta_{43} + \eta_{04} = 0$ \
$H_8$ & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & $\mu_2 + \eta_{43} + \eta_{04} = 0$ \
$H_9$ & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & $\mu_1 + \eta_{43} = 0$ \
$H_10$ & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 0 & $\mu_3 + \eta_{21} + \eta_{32} + \eta_{43} = 0$ \
$H_11$ & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & $\mu_2 + \eta_{32} + \eta_{43} = 0$ \
$H_12$ & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & $\mu_1 + \eta_{32} = 0$ \
$H_13$

In [54]:
inequalityKeys5 = collect(keys(s5))

113349-element Vector{Any}:
 Bool[1, 0, 1, 0, 0, 1, 0, 1, 1, 1  …  1, 0, 0, 1, 1, 0, 1, 0, 0, 1]
 Bool[1, 0, 0, 0, 0, 1, 0, 0, 0, 1  …  0, 1, 0, 0, 0, 1, 1, 0, 0, 1]
 Bool[0, 1, 1, 0, 1, 0, 1, 1, 0, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Bool[1, 1, 1, 0, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
 Bool[0, 1, 0, 0, 1, 1, 1, 0, 1, 0  …  1, 1, 0, 1, 1, 0, 1, 1, 0, 0]
 Bool[0, 1, 0, 0, 0, 1, 1, 1, 1, 1  …  1, 0, 1, 1, 0, 1, 1, 0, 0, 0]
 Bool[1, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 1, 1, 1, 0, 0, 0, 1, 1]
 Bool[0, 1, 0, 0, 1, 0, 0, 1, 1, 1  …  1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
 Bool[0, 0, 1, 1, 0, 1, 0, 0, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
 Bool[1, 0, 1, 1, 0, 1, 1, 0, 1, 0  …  1, 1, 0, 0, 0, 1, 1, 1, 1, 0]
 Bool[1, 1, 0, 1, 1, 0, 1, 1, 1, 0  …  1, 1, 0, 0, 0, 0, 1, 1, 0, 0]
 Bool[1, 0, 0, 0, 0, 1, 0, 0, 1, 1  …  0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
 Bool[1, 0, 1, 0, 0, 1, 0, 1, 1, 1  …  0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 ⋮
 Bool[1, 1, 0, 1, 1, 1, 0, 1, 1, 1  …  0, 1, 1, 1, 1, 0, 0, 0, 1, 1]
 Bo

In [56]:
for i=eachindex(inequalityKeys5)
    if ranks[i] == 10
        println(inequalityKeys5)
    end
end